# **CSCE 5218 / CSCE 4930 Deep Learning**

# **HW1a The Perceptron** (20 pt)


In [75]:
# Get the datasets
!wget http://huang.eng.unt.edu/CSCE-5218/test.dat
!wget http://huang.eng.unt.edu/CSCE-5218/train.dat


--2022-02-05 11:16:59--  http://huang.eng.unt.edu/CSCE-5218/test.dat
Resolving huang.eng.unt.edu (huang.eng.unt.edu)... 129.120.123.155
Connecting to huang.eng.unt.edu (huang.eng.unt.edu)|129.120.123.155|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2844 (2.8K)
Saving to: ‘test.dat.3’

test.dat.3          100%[===================>]   2.78K  --.-KB/s    in 0s      

2022-02-05 11:16:59 (385 MB/s) - ‘test.dat.3’ saved [2844/2844]

--2022-02-05 11:16:59--  http://huang.eng.unt.edu/CSCE-5218/train.dat
Resolving huang.eng.unt.edu (huang.eng.unt.edu)... 129.120.123.155
Connecting to huang.eng.unt.edu (huang.eng.unt.edu)|129.120.123.155|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11244 (11K)
Saving to: ‘train.dat.3’

train.dat.3         100%[===================>]  10.98K  --.-KB/s    in 0s      

2022-02-05 11:16:59 (68.6 MB/s) - ‘train.dat.3’ saved [11244/11244]



In [128]:
# Take a peek at the datasets
!head train.dat
!head test.dat

A1	A2	A3	A4	A5	A6	A7	A8	A9	A10	A11	A12	A13	
1	1	0	0	0	0	0	0	1	1	0	0	1	0
0	0	1	1	0	1	1	0	0	0	0	0	1	0
0	1	0	1	1	0	1	0	1	1	1	0	1	1
0	0	1	0	0	1	0	1	0	1	1	1	1	0
0	1	0	0	0	0	0	1	1	1	1	1	1	0
0	1	1	1	0	0	0	1	0	1	1	0	1	1
0	1	1	0	0	0	1	0	0	0	0	0	1	0
0	0	0	1	1	0	1	1	1	0	0	0	1	0
0	0	0	0	0	0	1	0	1	0	1	0	1	0
A1	A2	A3	A4	A5	A6	A7	A8	A9	A10	A11	A12	A13
1	1	1	1	0	0	1	1	0	0	0	1	1	0
0	0	0	1	0	0	1	1	0	1	0	0	1	0
0	1	1	1	0	1	1	1	1	0	0	0	1	0
0	1	1	0	1	0	1	1	1	0	1	0	1	0
0	1	0	0	0	1	0	1	0	1	0	0	1	0
0	1	1	0	0	1	1	1	1	1	1	0	1	0
0	1	1	1	0	0	1	1	0	0	0	1	1	0
0	1	0	0	1	0	0	1	1	0	1	1	1	0
1	1	1	1	0	0	1	1	0	0	0	0	1	0


### Build the Perceptron Model

You will need to complete some of the function definitions below.  DO NOT import any other libraries to complete this. 

In [129]:
import math
import itertools
import re


# Corpus reader, all columns but the last one are coordinates;
#   the last column is the label
def read_data(file_name):
    f = open(file_name, 'r')

    data = []
    # Discard header line
    next(f)
    for instance in f.readlines():
        if not re.search('\t', instance): continue
        instance = list(map(int, instance.strip().split('\t')))
        # Add a dummy input so that w0 becomes the bias
        w0 = 0
        instance = [-1] + instance
        data += [instance]
    return data


def dot_product(array1, array2):
    return sum(i[0] * i[1] for i in zip(array1, array2))

def sigmoid(x):
    return 1 / (1 + math.exp(-x))

# The output of the model, which for the perceptron is 
# the sigmoid function applied to the dot product of 
# the instance and the weights
def output(weight, instance):
    return sigmoid(dot_product(weight, instance))

# Predict the label of an instance; this is the definition of the perceptron
# you should output 1 if the output is >= 0.5 else output 0
def predict(weights, instance):
    return (1 if output(weights, instance) >= 0.5 else 0)


# Accuracy = percent of correct predictions
def get_accuracy(weights, instances):
    # You do not to write code like this, but get used to it
    correct = sum([1 if predict(weights, instance) == instance[-1] else 0
                   for instance in instances])
    return correct * 100 / len(instances)


# Train a perceptron with instances and hyperparameters:
#       lr (learning rate) 
#       epochs
# The implementation comes from the definition of the perceptron
#
# Training consists on fitting the parameters which are the weights
# that's the only thing training is responsible to fit
# (recall that w0 is the bias, and w1..wn are the weights for each coordinate)
#
# Hyperparameters (lr and epochs) are given to the training algorithm
# We are updating weights in the opposite direction of the gradient of the error,
# so with a "decent" lr we are guaranteed to reduce the error after each iteration.
def train_perceptron(instances, lr, epochs):

    #Weight Initialization
    weights = [0] * (len(instances[0])-1)

    for _ in range(epochs):
        for instance in instances:
            #Weighted Sum
            in_value = dot_product(weights, instance)
            #Activation Function
            output = sigmoid(in_value)
            #Performance Measure
            error = instance[-1] - output #error = real - predicted
            #Weight Training
            for i in range(0, len(weights)):
                weights[i] += lr * error * output * (1-output) * instance[i]

    return weights

## Run it

In [130]:
instances_tr = read_data("train.dat")
instances_te = read_data("test.dat")
lr = 0.005
epochs = 5
weights = train_perceptron(instances_tr, lr, epochs)
accuracy = get_accuracy(weights, instances_te)
print(f"#tr: {len(instances_tr):3}, epochs: {epochs:3}, learning rate: {lr:.3f}; "
      f"Accuracy (test, {len(instances_te)} instances): {accuracy:.1f}")

#tr: 400, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 68.0


## Questions

Answer the following questions. Include your implementation and the output for each question.



### Question 1

In `train_perceptron(instances, lr, epochs)`, we have the follosing code:
```
in_value = dot_product(weights, instance)
output = sigmoid(in_value)
error = instance[-1] - output
```

Why don't we have the following code snippet instead?
```
output = predict(weights, instance)
error = instance[-1] - output
```

#### TODO Add your answer here (text only)
We want to calculate error on the sigmoid value of the weighted inputs, as opposed to the output of the activation function (either 1 or 0). 



### Question 2
Train the perceptron with the following hyperparameters and calculate the accuracy with the test dataset.

```
tr_percent = [5, 10, 25, 50, 75, 100] # percent of the training dataset to train with
num_epochs = [5, 10, 20, 50, 100]              # number of epochs
lr = [0.005, 0.01, 0.05]              # learning rate
```

TODO: Write your code below and include the output at the end of each training loop (NOT AFTER EACH EPOCH)
of your code.The output should look like the following:
```
# tr:  20, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
# tr:  20, epochs:  10, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
# tr:  20, epochs:  20, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
[and so on for all the combinations]
```
You will get different results with different hyperparameters.

#### TODO Add your answer here (code and output in the format above) 


In [126]:
instances_tr = read_data("train.dat")
instances_te = read_data("test.dat")

tr_percent = [5, 10, 25, 50, 75, 100]
num_epochs = [5, 10, 25, 50, 75, 100]
lr_array = [0.005, 0.01, 0.05]

In [127]:
for rate in lr_array:
    for percent in tr_percent:
        for epoch in num_epochs:
            size = round(len(instances_tr) * percent / 100)
            pre_instances = instances_tr[0:size]
            weights = train_perceptron(pre_instances, rate, epoch)
            accuracy = get_accuracy(weights, instances_te)
            print(f"#train: {len(pre_instances):1}, epochs: {epoch:3}, learning rate: {rate:.3f}; "
                     f"Accuracy (test, {len(instances_te)} instances): {accuracy:.1f}")

#train: 20, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#train: 20, epochs:  10, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#train: 20, epochs:  25, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#train: 20, epochs:  50, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#train: 20, epochs:  75, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#train: 20, epochs: 100, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#train: 40, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#train: 40, epochs:  10, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#train: 40, epochs:  25, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#train: 40, epochs:  50, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#train: 40, epochs:  75, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#train: 40, epochs: 100, learning rate: 0.005; Accuracy (test, 100 instances

#train: 300, epochs:  75, learning rate: 0.050; Accuracy (test, 100 instances): 77.0
#train: 300, epochs: 100, learning rate: 0.050; Accuracy (test, 100 instances): 77.0
#train: 400, epochs:   5, learning rate: 0.050; Accuracy (test, 100 instances): 69.0
#train: 400, epochs:  10, learning rate: 0.050; Accuracy (test, 100 instances): 76.0
#train: 400, epochs:  25, learning rate: 0.050; Accuracy (test, 100 instances): 80.0
#train: 400, epochs:  50, learning rate: 0.050; Accuracy (test, 100 instances): 80.0
#train: 400, epochs:  75, learning rate: 0.050; Accuracy (test, 100 instances): 80.0
#train: 400, epochs: 100, learning rate: 0.050; Accuracy (test, 100 instances): 80.0


### Question 3
Write a couple paragraphs interpreting the results with all the combinations of hyperparameters. Drawing a plot will probably help you make a point. In particular, answer the following:
- A. Do you need to train with all the training dataset to get the highest accuracy with the test dataset?
- B. How do you justify that training the second run obtains worse accuracy than the first one (despite the second one uses more training data)?
   ```
#tr: 100, epochs:  20, learning rate: 0.050; Accuracy (test, 100 instances): 71.0
#tr: 200, epochs:  20, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
```
- C. Can you get higher accuracy with additional hyperparameters (higher than `80.0`)?
- D. Is it always worth training for more epochs (while keeping all other hyperparameters fixed)?

#### TODO: Add your answer here (code and text)

A. In all cases where the observed highest score of 80 is achieved, the full dataset was used. In fact, any score close to the highest used at least 50% of the dataset.

B. The higher learning rate causes the activation function to converge more quickly.

C. I predict that addditional hyperparameters will decrease the perceptron's accuracy, since they must also be optimized, requiring more time and compute.

D. No, in the case of this perceptron, convergence happened in ~25 epochs, where we achieved the highest accuracy score of 80. What plays a bigger role in accuracy is the amount of data the perceptron is exposed to.